<a href="https://colab.research.google.com/github/puckgroen/TM10007_PROJECT/blob/master/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

In [ ]:
# Import packages

# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn

# Classifiers
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from sklearn.impute import KNNImputer

In [0]:
# Data splitting

from hn.load_data import load_data
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns)}')
Y = data['label']
del data['label']

sss = model_selection.StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=0)
cv_4fold = model_selection.StratifiedKFold(n_splits=4)

count = 0 
for train_index, test_index in sss.split(data, Y):
    train_scores = []
    test_scores = []
    count += 1
    print(f'Iteration {count}')
    data_array = pd.DataFrame.from_dict(data) #Needed to be able to index this later 
    split_X_train = data_array.iloc[train_index]
    split_y_train = Y[train_index]
    split_X_test = data_array.iloc[test_index]
    split_y_test = Y[test_index]
    print(f'Training dataset size {len(split_X_train)}')
    print(f'Test dataset size {len(split_X_test)}')

    # Loop over the folds
    for training_index, validation_index in cv_4fold.split(split_X_train, split_y_train):
        X_validation = split_X_train.iloc[validation_index]
        y_validation = split_y_train[validation_index]
        X_train = split_X_train.iloc[training_index]
        y_train = split_y_train[training_index]
        print(f'Validation size in current fold = {len(X_validation)}')

The data has 159 features and 113 samples/subjects. The labels are given as either T12 (low) or T34 (high). 

The aim of this study is to predict the T-stage (high/low) in patients with H&N cancer based on features, extracted from CT. A good performance on this dataset would be above 70% mean accuracy.

Train data preprocessing

In [ ]:
# 1. Scaling (standard, min max, robust)

# Scale the data to be normal
scaler = preprocessing.StandardScaler()
# scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)


In [ ]:
# 2. Feature selection/extraction
# Create the Recursive Feature Elimination object and compute a cross-validated score.
svc = svm.SVC(kernel="linear")

# classifications
rfecv = feature_selection.RFECV(
    estimator=svc, step=1, 
    cv=model_selection.StratifiedKFold(4),
    scoring='roc_auc')
rfecv.fit(X_train_scaled, y_train)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()
print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
# 3. PCA
n_selected_features = rfecv.n_features_
n_samples = 113  # moeten we nog uit de data halen, is netter
n_components == min(n_samples, n_selected_features)
pca = decomposition.PCA(n_components)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)

seaborn.scatterplot(x=X_train_pca[:, 0], y=X_train_pca[:, 1], hue=y)

In [ ]:
# 4. Imputation missing data
missing_values=[0.0, 1.0]
for elem in missing_values:
    imputer = KNNImputer(missing_values=elem, n_neighbors=5, weights='uniform')
    X_train_imputed = imputer.fit_transform(X_train_pca)